<a href="https://colab.research.google.com/github/2303A51856/NLP/blob/main/PROJECT_NLP__1856.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers gensim tensorflow

In [ ]:
import pandas as pd

from google.colab import files
uploaded = files.upload()

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving Restaurant reviews.csv to Restaurant reviews (3).csv


,Restaurant,Reviewer,Review,Rating,Metadata,Time,Pictures,7514
0,Beyond Flavours,Rusha Chakraborty,"The ambience was good, food was quite good . h...",5,"1 Review , 2 Followers",5/25/2019 15:54,0,2447.0
1,Beyond Flavours,Anusha Tirumalaneedi,Ambience is too good for a pleasant evening. S...,5,"3 Reviews , 2 Followers",5/25/2019 14:20,0,NaN
2,Beyond Flavours,Ashok Shekhawat,A must try.. great food great ambience. Thnx f...,5,"2 Reviews , 3 Followers",5/24/2019 22:54,0,NaN
3,Beyond Flavours,Swapnil Sarkar,Soumen das and Arun was a great guy. Only beca...,5,"1 Review , 1 Follower",5/24/2019 22:11,0,NaN
4,Beyond Flavours,Dileep,Food is good.we ordered Kodi drumsticks and ba...,5,"3 Reviews , 2 Followers",5/24/2019 21:37,0,NaN


In [ ]:
import re
import nltk
import pandas as pd
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Download stopwords if not already downloaded
nltk.download('stopwords')

# Define stopwords
stop = set(stopwords.words('english'))

# Define text cleaning function
def clean_text(t):
    t = re.sub('[^a-zA-Z]', ' ', t)  # remove non-letter characters
    t = t.lower()                    # lowercase text
    t = t.split()                    # tokenize
    t = [w for w in t if w not in stop]  # remove stopwords
    return " ".join(t)

# Example DataFrame (remove this if df is already defined)
# df = pd.DataFrame({'Review': ['This movie was GREAT!', 'I did not like this film.']})

# Ensure Review column is string
df['Review'] = df['Review'].astype(str)

# Apply cleaning
df['clean'] = df['Review'].apply(clean_text)

# Show first few rows
print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


        Restaurant              Reviewer  \
0  Beyond Flavours     Rusha Chakraborty   
1  Beyond Flavours  Anusha Tirumalaneedi   
2  Beyond Flavours       Ashok Shekhawat   
3  Beyond Flavours        Swapnil Sarkar   
4  Beyond Flavours                Dileep   

                                              Review Rating  \
0  The ambience was good, food was quite good . h...      5   
1  Ambience is too good for a pleasant evening. S...      5   
2  A must try.. great food great ambience. Thnx f...      5   
3  Soumen das and Arun was a great guy. Only beca...      5   
4  Food is good.we ordered Kodi drumsticks and ba...      5   

                  Metadata             Time  Pictures    7514  \
0   1 Review , 2 Followers  5/25/2019 15:54         0  2447.0   
1  3 Reviews , 2 Followers  5/25/2019 14:20         0     NaN   
2  2 Reviews , 3 Followers  5/24/2019 22:54         0     NaN   
3    1 Review , 1 Follower  5/24/2019 22:11         0     NaN   
4  3 Reviews , 2 Followers  5/2

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,classification_report

# Drop rows where 'Rating' is NaN
df_cleaned = df.dropna(subset=['Rating']).copy()

X = df_cleaned['clean']
y = df_cleaned['Rating'] # Corrected column name to 'Rating'

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model_tfidf = SVC(kernel='linear')
model_tfidf.fit(X_train_tfidf,y_train)
pred = model_tfidf.predict(X_test_tfidf)

print("TF-IDF Accuracy:",accuracy_score(y_test,pred))
print(classification_report(y_test,pred))

TF-IDF Accuracy: 0.6287004515805319
              precision    recall  f1-score   support

           1       0.66      0.87      0.75       338
         1.5       0.00      0.00      0.00         2
           2       0.39      0.11      0.17       142
         2.5       0.00      0.00      0.00         7
           3       0.43      0.32      0.37       228
         3.5       0.00      0.00      0.00         5
           4       0.50      0.50      0.50       476
         4.5       0.00      0.00      0.00         9
           5       0.73      0.81      0.77       786

    accuracy                           0.63      1993
   macro avg       0.30      0.29      0.28      1993
weighted avg       0.60      0.63      0.60      1993



/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
from gensim.models import Word2Vec

sentences = [i.split() for i in df['clean']]
w2v = Word2Vec(sentences, vector_size=100, window=5, min_count=1)

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean'])

X_seq = tokenizer.texts_to_sequences(df['clean'])
maxlen = 100
X_pad = pad_sequences(X_seq, maxlen=maxlen)

y = df['Rating'] # Corrected column name to 'Rating'

X_train,X_test,y_train,y_test = train_test_split(X_pad,y,test_size=0.2,random_state=42)

In [ ]:
import numpy as np

vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((vocab_size,100))

for word, idx in tokenizer.word_index.items():
    if word in w2v.wv:
        embedding_matrix[idx] = w2v.wv[word]

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Sequential
# from tensorflow.keras.utils import to_categorical # Removed unnecessary import
import numpy as np
import pandas as pd # Import pandas for get_dummies

# One-hot encode the target variable
# Ensure y is defined correctly from previous steps, assuming y is from df['Rating']
y_encoded = pd.get_dummies(y).values

# Split data again after encoding y
# Make sure X_pad is defined from previous steps
X_train, X_test, y_train_encoded, y_test_encoded = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)


model_w2v = Sequential([
    Embedding(vocab_size,100,weights=[embedding_matrix],input_length=maxlen,trainable=False),
    Bidirectional(LSTM(128)),
    Dense(y_encoded.shape[1], activation='softmax') # Changed to match the number of unique classes after one-hot encoding
])

model_w2v.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model_w2v.summary()

model_w2v.fit(X_train,y_train_encoded,epochs=5,batch_size=32,validation_data=(X_test,y_test_encoded))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │     1,448,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,448,400 (5.53 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 1,448,400 (5.53 MB)

Epoch 1/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 69s 257ms/step - accuracy: 0.4205 - loss: 1.5015 - val_accuracy: 0.4982 - val_loss: 1.3084
Epoch 2/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 67s 270ms/step - accuracy: 0.4892 - loss: 1.3131 - val_accuracy: 0.5113 - val_loss: 1.2768
Epoch 3/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 63s 252ms/step - accuracy: 0.5000 - loss: 1.2972 - val_accuracy: 0.5334 - val_loss: 1.2489
Epoch 4/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 63s 251ms/step - accuracy: 0.5102 - loss: 1.2629 - val_accuracy: 0.5319 - val_loss: 1.2352
Epoch 5/5
249/249 ━━━━━━━━━━━━━━━━━━━━ 63s 254ms/step - accuracy: 0.5210 - loss: 1.2404 - val_accuracy: 0.5449 - val_loss: 1.2247


In [ ]:
from transformers import BertTokenizer, TFBertModel

tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased', from_pt=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Al

In [ ]:
def bert_encode(texts, tokenizer, max_len=100):
    input_ids, att = [], []

    for t in texts:
        enc = tokenizer.encode_plus(t, max_length=max_len, truncation=True, padding='max_length')
        input_ids.append(enc['input_ids'])
        att.append(enc['attention_mask'])
    return np.array(input_ids), np.array(att)

X_input_ids, X_att = bert_encode(df['clean'], tokenizer_bert)

X_train_ids, X_test_ids, X_train_att, X_test_att, y_train, y_test = train_test_split(
    X_input_ids, X_att, y, test_size=0.2, random_state=42)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls /content/drive/MyDrive


'2303A51856 batch 13 lab exam.ipynb'  'K.Abhinav_2303A51856_Team 97.pdf'
'2303A51856(Cisco) .pdf'	      'PPT_template(1) (1).pptx'
'Colab Notebooks'		      'PPT_template(1).pptx'
 DOC-20241122-WA0004..pdf	      'Problem_Statements(Hackathon) (1).docx'
'hackathon presentation.pdf'	      'Problem_Statements(Hackathon).docx'
'HACKATHON  problem H1-1.pdf'


In [8]:
!ls "/content/drive/MyDrive/Colab Notebooks"


'ADM PROJECT'	       'Lab 4.ipynb'		 PROJECT_NLP__1856.ipynb
 Lab01-Text.txt.ipynb   LAB_5_1856.ipynb	 Untitled0.ipynb
'Lab 02.ipynb'	       'Lab 5.ipynb'		 Untitled10.ipynb
'Lab 03.ipynb'	        LAB_6_1856.ipynb	 Untitled11.ipynb
 Lab04.ipynb	        LAB_7-1856.ipynb	 Untitled1.ipynb
 Lab06.ipynb	       'Lab 8 NLP_1856.ipynb'	 Untitled2.ipynb
 Lab07.ipynb	       'Lab exam.ipynb'		 Untitled3.ipynb
 Lab08.ipynb	       'Lab Exam q:1 .ipynb'	 Untitled4.ipynb
 Lab09.ipynb	       'Lab Exam q:2.ipynb'	 Untitled5.ipynb
 Lab10.ipynb	       'Lab Exam q:3.ipynb'	 Untitled6.ipynb
'LAB 1.1856'	       'Lab Exam q:4.ipynb'	 Untitled7.ipynb
 LAB_2_1856.ipynb      'Lab Exam q:5.ipynb'	 Untitled8.ipynb
 LAB_3_1856.ipnyb      'Lab Exam q:6.ipynb'	 Untitled9.ipynb
 LAB_4_1856.ipynb       NLP_PROJECT_1856.ipynb


In [12]:
from google.colab import runtime
import json
import requests

# The notebook_metadata() function is deprecated and not available.
# There is no direct replacement for getting the notebook path this way.
# If you need to access files in your Drive, use paths like '/content/drive/MyDrive/...'

# def get_notebook_path():
#     info = json.loads(runtime.notebook_metadata())
#     return info.get('path', 'Unknown path')

# print(get_notebook_path())